# Contradictory, My Dear Watson
Hello, everyone! In this my new notebook we are going to solve the ["Contradictory, My Dear Watson"](https://www.kaggle.com/c/contradictory-my-dear-watson) problem. 

> "…when you have eliminated the impossible, whatever remains, however improbable, must be the truth" - Sir Arthur Conan Doyle


#### References
For this notebook I would like to say thank you some authors for their notebooks that have inspired me to write own notebook:
1. [ANA SOFIA UZSOY. Tutorial Notebook](https://www.kaggle.com/anasofiauzsoy/tutorial-notebook)

#### Natural language processing (NLP) 
Natural language processing (NLP) has grown increasingly elaborate over the past few years. Machine learning models tackle question answering, text extraction, sentence generation, and many other complex tasks. But, can machines determine the relationships between sentences, or is that still left to humans? If NLP can be applied between sentences, this could have profound implications for fact-checking, identifying fake news, analyzing text, and much more.

![detective](https://cdn0.iconfinder.com/data/icons/streamline-emoji-1/48/192-detective-2-512.png)

# 1. Problem Formulation
Our task is to create an NLI model that assigns labels of 0, 1, or 2 (corresponding to entailment, neutral, and contradiction) to pairs of premises and hypotheses. To make things more interesting, the train and test set include text in fifteen different languages! 

# 2. Import libraries

In [ ]:
import os
import numpy as np
import pandas as pd
from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [ ]:
os.environ["WANDB_API_KEY"] = "0" # to silence warning

# 3. Turn on TPU
TPUs are powerful hardware accelerators specialized in deep learning tasks, including Natural Language Processing. Kaggle provides all users TPU Quota at no cost, which you can use to explore this competition. 

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

# 3. Read data

In [ ]:
df_train = pd.read_csv("../input/contradictory-my-dear-watson/train.csv")
df_test = pd.read_csv("../input/contradictory-my-dear-watson/test.csv")

In [ ]:
df_train.head(7) # check data (first 7 rows)

In [ ]:
df_test.head(7) # check data (first 7 rows)

In [ ]:
df_train.shape # check data

In [ ]:
df_test.shape # check data

Let's check some rows more in more detailed way. As example, I'll use the row with 0 index.

- **Premise** - a previous statement or proposition from which another is inferred or follows as a conclusion.
- **Hypothesis** - a supposition or proposed explanation made on the basis of limited evidence as a starting point for further investigation.

In [ ]:
# "premise" column
df_train["premise"].values[0]

In [ ]:
# "hypothesis" column
df_train["hypothesis"].values[0]

# 3. Vizualize data
Here we can build some plots and visualizations.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(20,10))
count_classes = df_train['language'].value_counts() # count sentenses in each language
plt.title("Languages in df_train".upper())
colors = ['#ff9999','#66b3ff','#99ff99',
          '#ffcc99', '#ffccf9', '#ff99f8', 
          '#ff99af', '#ffe299', '#a8ff99',
          '#cc99ff', '#9e99ff', '#99c9ff',
          '#99f5ff', '#99ffe4', '#99ffaf']

plt.pie(count_classes, 
        labels = count_classes.index, 
        autopct='%1.1f%%',
        shadow=True, 
        startangle=90, 
        colors=colors)

plt.show()

After that, we can build some more stats:

In [ ]:
def plot_stats(df, column, ax, color, angle):
    """ PLOT STATS OF DIFFERENT COLUMNS """
    count_classes = df[column].value_counts()
    ax = sns.barplot(x=count_classes.index, y=count_classes, ax=ax, palette=color)
    ax.set_title(column.upper(), fontsize=18)
    for tick in ax.get_xticklabels():
        tick.set_rotation(angle)

Here we can see stats for **"label"**:

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(15,5))
fig.autofmt_xdate()
plot_stats(df_train, "label", axes, "viridis", 45)
plt.show()

# 4. Download pretrained models
To start out, we can use a pretrained model. Here, we'll use a multilingual BERT model from huggingface. For more information about BERT, see [here](https://github.com/google-research/bert/blob/master/multilingual.md).

## 4.1 Download tokenizer
Tokenizers turn sequences of words into arrays of numbers. 

In [ ]:
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
len(tokenizer.vocab) # check the vocabulary size

Let's check the tokenizer. As example, we can take the sentence **"you know they can't really defend themselves"**.

The model expects its two inputs sentences to be concatenated together. This input is expected to start with a [CLS] "This is a classification problem" token, and each sentence should end with a [SEP] "Separator" token.

In [ ]:
def encode_sentence(s):
    """ ENCODE SENTENCES WITH TOKENIZER"""
    tokens = list(tokenizer.tokenize(s))
    tokens.append('[SEP]')
    return tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
encode_sentence("you know they can't really defend themselves")

## 4.2 BERT
[BERT](https://huggingface.co/transformers/model_doc/bert.html#tfbertmodel) **uses three kind of input data** - input word IDs, input masks, and input type IDs.

These allow the model to know that **the premise and hypothesis are distinct sentences**, and also to ignore any padding from the tokenizer.

We add a [CLS] token to denote **the beginning of the inputs**, and a [SEP] token to denote the separation between **the premise and the hypothesis**. 

We also **need to pad** all of the inputs to be the same size.

Now, we're going to **encode all of our premise/hypothesis pairs** for input into BERT.

In [ ]:
def bert_encode(hypotheses, premises, tokenizer):
    """ ENCODE DATA FOR BERT"""
    num_examples = len(hypotheses)
    print("num_examples = ", num_examples)
    sentence1 = tf.ragged.constant([encode_sentence(s) for s in np.array(hypotheses)])
    print("sentence1.shape = ", sentence1.shape)
    sentence2 = tf.ragged.constant([encode_sentence(s) for s in np.array(premises)])
    print("sentence2.shape = ", sentence2.shape)
    cls_ = [tokenizer.convert_tokens_to_ids(['[CLS]'])] * sentence1.shape[0]
    input_word_ids = tf.concat([cls_, sentence1, sentence2], axis=-1)
    print("input_word_ids.shape = ", input_word_ids.shape)
    # 300 - as my example
    # because we have train_input (12120; 259), test_input (5159; 234)
    # and shape[1] should be the same in each dataset
    # that is why we creating (xxx; 300) shape in to_tensor() functions  
    input_mask = tf.ones_like(input_word_ids).to_tensor(shape=(input_word_ids.shape[0], 300)) 
    print("input_mask.shape = ", input_mask.shape)
    
    type_cls = tf.zeros_like(cls_)
    type_s1 = tf.zeros_like(sentence1)
    type_s2 = tf.ones_like(sentence2)
    
    input_type_ids = tf.concat([type_cls, type_s1, type_s2], axis=-1).to_tensor(shape=(input_word_ids.shape[0], 300))
    
    inputs = {'input_word_ids': input_word_ids.to_tensor(shape=(input_word_ids.shape[0], 300)),
              'input_mask': input_mask,
              'input_type_ids': input_type_ids}
    print()
    
    return inputs
    

In [ ]:
# encode data
train_input = bert_encode(df_train["premise"].values, df_train["hypothesis"].values, tokenizer)
test_input = bert_encode(df_test["premise"].values, df_test["hypothesis"].values, tokenizer)

In [ ]:
train_input # check train input

In [ ]:
test_input # check test input

# 5. Train Neural Network Model
Now, we can incorporate the BERT transformer into a Keras Functional Model. 

In [ ]:
max_len = train_input["input_word_ids"].shape[1]

def create_model():
    """ BUILD MODEL """
    bert_encoder = TFBertModel.from_pretrained(model_name)
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    input_type_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_type_ids")

    embedding = bert_encoder([input_word_ids, input_mask, input_type_ids])[0]
    output = tf.keras.layers.Dense(3, activation='softmax')(embedding[:,0,:])

    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=output)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
with strategy.scope():
    model = create_model()
    model.summary()

In [ ]:
model_history = model.fit(train_input, 
                          df_train["label"].values, 
                          epochs = 20, 
                          verbose = 1,
                          batch_size = 128, 
                          validation_split = 0.2)

Also, we can build **training history**:

In [ ]:
def plot_NN_history(model_history, suptitle):
    # plot data
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,6))
    fig.suptitle(suptitle, fontsize=18)
    
    axes[0].plot(model_history.history['accuracy'], label='train accuracy', color='g', axes=axes[0])
    axes[0].plot(model_history.history['val_accuracy'], label='val accuracy', color='r', axes=axes[0])
    axes[0].set_title("Model Accuracy", fontsize=16) 
    axes[0].legend(loc='upper left')

    axes[1].plot(model_history.history['loss'], label='train loss', color='g', axes=axes[1])
    axes[1].plot(model_history.history['val_loss'], label='val loss', color='r', axes=axes[1])
    axes[1].set_title("Model Loss", fontsize=16) 
    axes[1].legend(loc='upper left')

    plt.show()

In [ ]:
plot_NN_history(model_history, "BERT")

# 6. Test Neural Network

In [ ]:
def calculate_results(y_true, y_pred):
    """ CALCULATE RESULTS"""
    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    # Calculate model precision, recall and f1 score using "weighted" average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    model_results = {"accuracy": model_accuracy,
                     "precision": model_precision,
                     "recall": model_recall,
                     "f1": model_f1}
    return model_results

In [ ]:
# get the probabilities
y_prob = model.predict(test_input)
# get the classes
y_hat = y_prob.argmax(axis=-1)

# 7. Make submission

In [ ]:
submission = df_test.id.copy().to_frame()
submission['prediction'] = y_hat
submission.head() # check submission
submission.to_csv("submission.csv", index = False) # save file

# 8. Conclusion
Thank you for reading my new article! Hope, you liked it and it was interesting for you! There are some more my articles:
* [SMS spam with NBC | NLP | sklearn](https://www.kaggle.com/maricinnamon/sms-spam-with-nbc-nlp-sklearn)
* [House Prices Regression sklearn](https://www.kaggle.com/maricinnamon/house-prices-regression-sklearn)
* [Automobile Customer Clustering (K-means & PCA)](https://www.kaggle.com/maricinnamon/automobile-customer-clustering-k-means-pca)
* [Credit Card Fraud detection sklearn](https://www.kaggle.com/maricinnamon/credit-card-fraud-detection-sklearn)
* [Market Basket Analysis for beginners](https://www.kaggle.com/maricinnamon/market-basket-analysis-for-beginners)
* [Neural Network for beginners with keras](https://www.kaggle.com/maricinnamon/neural-network-for-beginners-with-keras)
* [Fetal Health Classification for beginners sklearn](https://www.kaggle.com/maricinnamon/fetal-health-classification-for-beginners-sklearn)
* [Retail Trade Report Department Stores (LSTM)](https://www.kaggle.com/maricinnamon/retail-trade-report-department-stores-lstm)